In [1]:
# ... (cabeçalho permanece o mesmo)
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.data_loader import load_data
from src.preprocessor import DataPreprocessor

# %% [markdown]
# ## 1. Carregar e Pré-processar Dados
# %%
# Carregar dados
print("Carregando dados...")
df_raw = load_data()
print(f"Dados brutos carregados: {df_raw.shape[0]} registros, {df_raw.shape[1]} colunas")

# Verificar colunas disponíveis
print("\nColunas disponíveis nos dados brutos:")
print(df_raw.columns.tolist())

# Pré-processar dados
print("\nPré-processando dados...")
preprocessor = DataPreprocessor()
df = preprocessor.preprocess(df_raw)
print(f"Dados pré-processados: {df.shape[0]} registros, {df.shape[1]} colunas")

# Verificar colunas disponíveis após pré-processamento
print("\nColunas disponíveis após pré-processamento:")
print(df.columns.tolist())

# %% [markdown]
# ## 2. Análise Temporal de Casos e Clima
# %%
if 'data' in df.columns:
    # Agrupar por mês
    df_monthly = df.groupby([pd.Grouper(key='data', freq='M'), 'municipio']).agg({
        'casos_dengue': 'sum',
        'casos_zika': 'sum',
        'casos_chikungunya': 'sum',
        'temperatura': 'mean',
        'umidade': 'mean',
        'precipitacao': 'sum'
    }).reset_index()
    
    # Plotar casos
    plt.figure(figsize=(14, 10))
    for i, disease in enumerate(['dengue', 'zika', 'chikungunya'], 1):
        plt.subplot(3, 1, i)
        for city in df['municipio'].unique():
            city_data = df_monthly[df_monthly['municipio'] == city]
            plt.plot(city_data['data'], city_data[f'casos_{disease}'], label=city)
        plt.title(f'Casos de {disease.capitalize()} por Mês')
        plt.ylabel('Casos')
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Plotar variáveis climáticas
    plt.figure(figsize=(14, 10))
    for i, var in enumerate(['temperatura', 'umidade', 'precipitacao'], 1):
        plt.subplot(3, 1, i)
        for city in df['municipio'].unique():
            city_data = df_monthly[df_monthly['municipio'] == city]
            plt.plot(city_data['data'], city_data[var], label=city)
        plt.title(f'{var.capitalize()} Média Mensal')
        plt.ylabel(var.capitalize())
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Dados insuficientes para análise temporal")

# %% [markdown]
# ## 3. Correlação entre Clima e Casos
# %%
if 'temperatura' in df.columns:
    # Calcular correlação
    corr_matrix = df[['temperatura', 'umidade', 'precipitacao', 
                      'casos_dengue', 'casos_zika', 'casos_chikungunya']].corr()
    
    # Plotar mapa de calor
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f")
    plt.title('Correlação entre Variáveis Climáticas e Casos de Arboviroses')
    plt.tight_layout()
    plt.show()
    
    # Atrasos de precipitação
    if 'precipitacao' in df.columns:
        plt.figure(figsize=(14, 8))
        for i, disease in enumerate(['dengue', 'zika', 'chikungunya'], 1):
            plt.subplot(3, 1, i)
            
            # Calcular correlação com lags de precipitação
            lags = range(0, 31, 5)  # 0 a 30 dias
            correlations = []
            for lag in lags:
                corr = df[f'casos_{disease}'].corr(df['precipitacao'].shift(lag))
                correlations.append(corr)
            
            plt.plot(lags, correlations, 'o-')
            plt.title(f'Correlação entre Precipitação (lag) e Casos de {disease.capitalize()}')
            plt.xlabel('Dias de Atraso (lag)')
            plt.ylabel('Correlação')
            plt.grid(True)
        
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ Dados insuficientes para análise climática")

# ... (resto do notebook permanece similar)

c:\Users\ja\anaconda3\lib\site-packages\pandas\_testing.py:24: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  import pandas._libs.testing as _testing


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations